In [7]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json
from datetime import datetime 

# Load API key from .env file
load_dotenv()
coinapi_key = os.getenv('api_key')

exchange_id = "DERIBIT"

# CoinAPI endpoint for symbols metadata
url = f"https://rest.coinapi.io/v1/options/{exchange_id}/current"

#payload = {}
headers = {
  'Accept': 'text/plain',
  'X-CoinAPI-Key': coinapi_key
}

#response = requests.request("GET", url, headers=headers, data=payload)
response = requests.request("GET", url, headers=headers)

#print(response.text)

# Check for successful response
if response.status_code == 200:
    data = response.json()
    
    # Filter for BTC/USD call options
    filtered_calls = []
    for option in data:
        if option.get("asset_id_base") == "BTC" and option.get("asset_id_quote") == "USD":
            underlying_price = option.get("underlying_price")
            time_expiration = option.get("time_expiration")

            for strike in option.get("strikes", []):
                if "call" in strike:
                    call_data = {
                        "symbol_id": strike["call"]["symbol_id"],
                        "time_exchange": strike["call"]["time_exchange"],
                        "time_coinapi": strike["call"]["time_coinapi"],
                        "strike_price": strike["strike_price"],
                        "ask_price": strike["call"]["ask_price"],
                        "ask_size": strike["call"]["ask_size"],
                        "bid_price": strike["call"]["bid_price"],
                        "bid_size": strike["call"]["bid_size"],
                        "underlying_price": underlying_price,
                        "time_expiration": time_expiration
                    }
                    filtered_calls.append(call_data)

    # Convert to DataFrame for better visualization
    df_calls = pd.DataFrame(filtered_calls)
    pd.set_option('display.width', 1000)
    pd.set_option('display.max_columns', None)
    print(df_calls)

else:
    print(f"Error: {response.status_code}, {response.text}")


                               symbol_id                 time_exchange                  time_coinapi  strike_price  ask_price  ask_size  bid_price  bid_size  underlying_price               time_expiration
0    DERIBIT_OPT_BTC_USD_250425_104000_C  2025-02-04T10:41:32.4584885Z  2025-02-04T10:41:32.4585111Z      104000.0     0.0920       0.8     0.0910       2.3      98660.333262  2025-04-25T08:00:00.0000000Z
1    DERIBIT_OPT_BTC_USD_250425_102000_C  2025-02-04T10:41:37.1103509Z  2025-02-04T10:41:37.1103697Z      102000.0     0.1000      10.3     0.0990       1.3      98660.333262  2025-04-25T08:00:00.0000000Z
2    DERIBIT_OPT_BTC_USD_250425_115000_C  2025-02-04T10:41:37.0453194Z  2025-02-04T10:41:37.0453519Z      115000.0     0.0580       0.7     0.0570      11.2      98660.333262  2025-04-25T08:00:00.0000000Z
3    DERIBIT_OPT_BTC_USD_250425_120000_C  2025-02-04T10:41:31.8114647Z  2025-02-04T10:41:31.8114853Z      120000.0     0.0470       6.6     0.0460      28.9      98660.333262  2025

# Data Preparation

### Calculate 
### Market Price using ((ask price + bid price) / 2)
### Maturity(Years) using time_expiration and time_exchange
### Risk-Free Rate is gotten from OpenBB


### Maturity Years

In [8]:
df_calls = pd.DataFrame(filtered_calls)

#convert time columns to datetime
df_calls['time_exchange'] = pd.to_datetime(df_calls['time_exchange'])
df_calls['time_expiration'] = pd.to_datetime(df_calls['time_expiration'])

#Calculate the maturity in years
df_calls['time_to_maturity'] = (df_calls['time_expiration'] - df_calls['time_exchange']).dt.total_seconds() / (365.25 * 24 * 3600)
df_calls['time_to_maturity']

0      0.218721
1      0.218721
2      0.218721
3      0.218721
4      0.218722
         ...   
373    0.007907
374    0.007906
375    0.007955
376    0.007906
377    0.007906
Name: time_to_maturity, Length: 378, dtype: float64

### Market Price

In [9]:
df_calls['market_price'] = (df_calls['ask_price'] + df_calls['bid_price']) / 2
df_calls['market_price']

0      0.09150
1      0.09950
2      0.05750
3      0.04650
4      0.14900
        ...   
373    0.00025
374    0.00420
375    0.00015
376    0.00220
377    0.00110
Name: market_price, Length: 378, dtype: float64

In [10]:
# jupyter notebook


In [ ]:


import sys
import os

parent_dir = os.path.abspath("..")

sys.path.append(parent_dir)

from treasury_yield.risk_free_rate import risk_free_rate

df_calls["risk_free_rate"] = risk_free_rate


In [12]:
# Selecting relevant columns
calibration_data = df_calls[['strike_price', 'time_to_maturity', 'market_price', 'underlying_price', 'risk_free_rate']]

print(calibration_data)

     strike_price  time_to_maturity  market_price  underlying_price  risk_free_rate
0        104000.0          0.218721       0.09150      98660.333262         0.04543
1        102000.0          0.218721       0.09950      98660.333262         0.04543
2        115000.0          0.218721       0.05750      98660.333262         0.04543
3        120000.0          0.218721       0.04650      98660.333262         0.04543
4         92000.0          0.218722       0.14900      98660.333262         0.04543
..            ...               ...           ...               ...             ...
373      114000.0          0.007907       0.00025      98660.333262         0.04543
374      104000.0          0.007906       0.00420      98660.333262         0.04543
375      116000.0          0.007955       0.00015      98660.333262         0.04543
376      106000.0          0.007906       0.00220      98660.333262         0.04543
377      108000.0          0.007906       0.00110      98660.333262         